### Cable Corridor Computation


In [ ]:
# ignore the nopython dependency warning, as it cant be fixed with current dependencies
import warnings

warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

# Package Imports
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Polygon, Point
import pickle

# Importing my modules
from src.main import global_vars
from src.main.classes_geometry_objects import LineString_3D, Point_3D
from src.main import optimization_execution
from src.main import geometry_utilities, mechanical_computations

# Interface modules
from ipywidgets import VBox, HBox
from src.main.interface import interactive_cr_selection

In [2]:
%load_ext autoreload
%autoreload 2

Instructions for usage:

1. Load the libraries above
2. Execute the cells in "Load Resources" - this will take several minutes
3. Run the Interface in "Plotting"


#### Preprocessing Archive


In [4]:
# # Load, preprocess and save the dataframe with the corresponding number - only needed to do once
# data_loading.load_and_preprocess_raw_data(data_to_load=3)
# tree_gdf, height_gdf = data_loading.load_processed_gdfs(data_to_load=3)
# # create coordinates for each dataframe with the corresponding upper and lower anchors
# coords = []
# fig, ax = plt.subplots()
# ax.set_xlabel("X coordinate", fontsize=15)
# ax.set_ylabel("Y coordinate", fontsize=15)
# ax.scatter(tree_gdf["x"], tree_gdf["y"])

# cid = fig.canvas.mpl_connect(
#     "button_press_event", lambda event: plotting_2d.onclick(event, coords)
# )

# from src.main import classes_cable_road_computation
# from importlib import reload

# reload(classes_cable_road_computation)

# from src.main import classes_cable_road_computation
# import types


# from src.main.cable_road_computation_main import cable_road_computation
# from src.main import cable_road_computation

# forest_area_3.compute_cable_road = types.MethodType(
#     classes_cable_road_computation.forest_area.compute_cable_road, forest_area_3
# )

# from src.main.classes_geometry_objects import LineString_3D, Point_3D
# from src.main import geometry_utilities, mechanical_computations

# cable_road_object = forest_area_3.line_gdf.iloc[0]["Cable Road Object"]

# loaded_index = len(cable_road_object.points_along_line) // 2

# # get the centroid, lines and angles of the two CRs, once tensioned, once empty
# line_centroid_loaded_cr = (
#     mechanical_computations.get_line_3d_from_cr_startpoint_to_centroid(
#         cable_road_object,
#         move_towards_start_point=False,
#         sloped=True,
#         index=loaded_index,
#     )
# )


# def compute_line_from_top_to_bottom(support_object):
#     xy_point = support_object.xy_location
#     xy_floor = support_object.floor_height
#     xy_height = support_object.total_height

#     # piece together the triple from the xy coordinates and the z (height)
#     start_point_xyz = (
#         xy_point.coords[0][0],
#         xy_point.coords[0][1],
#         xy_height,
#     )
#     end_point_xyz = (
#         xy_point.coords[0][0],
#         xy_point.coords[0][1],
#         xy_floor,
#     )

#     end_point_xyz = Point_3D(*end_point_xyz)
#     start_point_xyz = Point_3D(*start_point_xyz)

#     return LineString_3D(start_point_xyz, end_point_xyz)


# line_support_from_bottom_to_top = compute_line_from_top_to_bottom(
#     cable_road_object.start_support
# )

# angle_between_support_cr = geometry_utilities.angle_between_3d_lines(
#     line_centroid_loaded_cr, line_support_from_bottom_to_top
# )

# print(angle_between_support_cr)

# # print the coordinates of the centroid and the support lines
# print(line_centroid_loaded_cr.start_point.xyz)
# print(line_support_from_bottom_to_top.end_point.xyz)

# # store forest_area_3
# output = open("forest_area_3_10_2024.pkl", "wb")
# pickle.dump(forest_area_3, output)

# # # new forest area as of march
# # output = open("forest_area_3_03_2024.pkl", "wb")
# # pickle.dump(forest_area_3, output)

# # from src.main import classes_cable_road_computation

# # create a class instance of the forest area with all the necessary attributes
# forest_area_3_25_04_2024 = classes_cable_road_computation.forest_area(
#     forest_area_3.tree_gdf, forest_area_3.height_gdf, extra_geometry_gpd
# )

# # store optimization_results
# output = open("optimization_results_10_2024.pkl", "wb")
# pickle.dump(optimization_results, output, protocol=4)

# # load the optimization_results
# pkl_file = open("optimization_results.pkl", "rb")
# optimization_results = pickle.load(pkl_file)
# results_df, optimization_result_list, model_list = optimization_results

# # and the forest_area_3
# pkl_file = open("forest_area_3.pkl", "rb")
# forest_area_3 = pickle.load(pkl_file)

# output = open("forest_area_3_10_2024.pkl", "wb")
# pickle.dump(forest_area_3, output)

#### Load Resources


In [ ]:
# load the optimization_results
pkl_file = open("optimization_results_10_2024.pkl", "rb")
optimization_results = pickle.load(pkl_file)
results_df, optimization_result_list, model_list = optimization_results

# and the forest_area_3
pkl_file = open("forest_area_3_05_2024.pkl", "rb")
forest_area_3 = pickle.load(pkl_file)

# forest_area_3.compute_line_costs()
kdtree = global_vars.init(forest_area_3.height_gdf)

In [ ]:
# manually insert those coordinates
# recorded coordinates for the given areas
def compute_geometry():
    downhill_anchors = Polygon(
        [
            Point(xy)
            for xy in [
                (-123.88268627035482, 83.05479790035122),
                (-74.89307684787903, 138.5299549848453),
                (-21.10056454084676, 161.97821725767267),
                (-6.051468835903222, 148.2524051955298),
                (-60.16417466857257, 116.79741921978575),
                (-115.23746107815322, 64.75371515082742),
            ]
        ]
    )
    uphill_anchors = Polygon(
        [
            Point(xy)
            for xy in [
                (-55.04107825837903, -15.31352187833928),
                (-17.578435758838708, -17.601157222029755),
                (4.194723984483858, -1.5877098161964227),
                (30.450593086725803, -30.18315161232738),
                (-11.49475877173387, -41.04941949485714),
                (-58.5632070403871, -38.189875315244045),
            ]
        ]
    )
    # road = LineString([(-57.282432937838706, -17.029248386107135), (-34.86888614324194, -21.604519073488092), (-15.337081079379033, -15.31352187833928), (-0.2879853744354932, -7.878707011345227), (12.51975565104837, 6.990922722642864), (19.884206740701615, 21.86055245663097)])
    # road = LineString([(-59.52378761729838, -12.453977698726185), (-41.91314370725806, -13.597795370571419), (-18.218822810112897, -10.16634235503571), (3.2341434075725886, 0.12801669157144602), (15.401497381782264, 23.576278964398824), (17.00246500996775, 26.43582314401192)])
    road = LineString(
        [
            (-89.6286010742188, -24.4624996185303),
            (-69.618896484375, -20.364200592041),
            (-41.314998626709, -13.9251003265381),
            (-29.2644004821777, -8.47910022735596),
            (-20.2950000762939, -3.67030000686646),
            (-15.375, -0.148599997162819),
            (-6.49030017852783, 2.74860000610352),
            (0.943400025367737, 3.86450004577637),
            (15.1605997085571, -0.302300006151199),
            (29.3377990722656, -3.47709989547729),
        ]
    )

    inner_forest = Polygon(
        [
            Point(xy)
            for xy in [
                (-105.43005197808363, 62.73802056488026),
                (-10.557896233758669, 146.34023494632802),
                (13.160142702322588, 51.07259530235265),
                (8.890895693827957, 9.27148811162877),
                (-55.62217021231302, -10.17088732591722),
                (-81.71201304200238, 18.992675830401765),
            ]
        ]
    )

    extra_geometry_gpd = gpd.GeoDataFrame(
        pd.DataFrame(
            {
                "name": ["uphill_anchors", "road", "downhill_anchors", "inner_forest"],
                "geometry": [uphill_anchors, road, downhill_anchors, inner_forest],
            }
        )
    )

    # set the name as index to easily select the geometry
    extra_geometry_gpd.set_index("name", inplace=True)
    extra_geometry_gpd.loc["road"]

    return extra_geometry_gpd


extra_geometry_gpd = compute_geometry()


def compute_line_from_top_to_bottom(support_object):
    xy_point = support_object.xy_location
    xy_floor = support_object.floor_height
    xy_height = support_object.total_height

    # piece together the triple from the xy coordinates and the z (height)
    start_point_xyz = (
        xy_point.coords[0][0],
        xy_point.coords[0][1],
        xy_height,
    )
    end_point_xyz = (
        xy_point.coords[0][0],
        xy_point.coords[0][1],
        xy_floor,
    )

    end_point_xyz = Point_3D(*end_point_xyz)
    start_point_xyz = Point_3D(*start_point_xyz)

    return LineString_3D(start_point_xyz, end_point_xyz)


def compute_angle_between_support_and_cr(line_gdf, support_type):
    for index, row in line_gdf.iterrows():
        cable_road_object = row["Cable Road Object"]
        loaded_index = len(cable_road_object.points_along_line) // 2

        line_centroid_loaded_cr = (
            mechanical_computations.get_line_3d_from_cr_startpoint_to_centroid(
                cable_road_object,
                move_towards_start_point=False if support_type == "start" else True,
                sloped=True,
                index=loaded_index,
            )
        )

        support_object = getattr(cable_road_object, f"{support_type}_support")
        line_support_from_bottom_to_top = compute_line_from_top_to_bottom(
            support_object
        )

        angle_between_support_cr = geometry_utilities.angle_between_3d_lines(
            line_centroid_loaded_cr, line_support_from_bottom_to_top
        )

        line_gdf.loc[index, f"angle_between_{support_type}_support_and_cr"] = (
            angle_between_support_cr
        )

    return line_gdf


def compute_angles_between_supports_and_cr(line_gdf):
    line_gdf = compute_angle_between_support_and_cr(line_gdf, "start")
    line_gdf = compute_angle_between_support_and_cr(line_gdf, "end")
    return line_gdf


def compute_cable_corridors(forest_area_3):
    # # and the forest_area_3
    # pkl_file = open("forest_area_3_05_2024.pkl", "rb")
    # forest_area_3 = pickle.load(pkl_file)

    # Readjusting the mdm computation and the corresponding euler knicklastforces
    # forest_area_3 = forest_area_3_25_04_2024

    forest_area_3.tree_gdf["middle_height"] = forest_area_3.tree_gdf[
        "diameter_series"
    ].apply(lambda x: len(x) // 2)
    # and the corresponding diameter there
    forest_area_3.tree_gdf["mdm"] = forest_area_3.tree_gdf.apply(
        lambda x: x["diameter_series"][x["middle_height"]], axis=1
    )

    # get the euler forces
    list_of_euler_max_force_lists = []
    for index, row in forest_area_3.tree_gdf.iterrows():
        temp_list = [
            mechanical_computations.euler_knicklast(row["mdm"], height * 2)
            for height in row["height_series"]
        ]
        list_of_euler_max_force_lists.append(temp_list)

    forest_area_3.tree_gdf["max_supported_force_series"] = list_of_euler_max_force_lists

    forest_area_3.compute_cable_road()

    forest_area_3.line_gdf = compute_angles_between_supports_and_cr(
        forest_area_3.line_gdf
    )

    forest_area_3.compute_line_costs()

    return forest_area_3


forest_area_3 = compute_cable_corridors(forest_area_3)


def optimization_main_loop(forest_area_gdf):
    """Main loop for the optimization algorithm"""

    optimization_result_list = []
    model_list = []

    # first the basis optimization
    print("Starting basis optimization")
    (
        model_list,
        optimization_result_list,
    ) = optimization_execution.soo_optimization_augmecon(
        model_list, optimization_result_list, forest_area_gdf
    )

    # then the manual weights optimization
    print("Starting manual weights optimization")
    optimization_result_list = optimization_execution.soo_optimization_manual_weights(
        model_list, optimization_result_list, forest_area_gdf
    )

    # then the NSGA II optimization
    print("Starting NSGA II optimization")
    optimization_result_list = optimization_execution.nsga_optimization(
        model_list, optimization_result_list, forest_area_gdf
    )

    # create a result df to save the results
    results_df = optimization_execution.create_results_df(
        optimization_result_list, model_list[0], forest_area_gdf
    )

    # # create ranges for the different objectives based on the results
    # (
    #     ecological_range,
    #     ecological_step,
    #     ergonomics_range,
    #     ergonomics_step,
    # ) = optimization_execution.compute_augmecon_ranges(results_df)

    # # conduct the augmecon optimization
    # print("Starting augmecon optimization")
    # optimization_result_list = optimization_execution.augmecon_optimization(
    #     optimization_result_list,
    #     ecological_range,
    #     ecological_step,
    #     ergonomics_range,
    #     ergonomics_step,
    #     forest_area_gdf,
    # )

    # and finally the expert layout optimization
    # print("Starting expert layout optimization")
    # optimization_result_list = optimization_execution.expert_layout_optimization(
    #     optimization_result_list, model_list[0], forest_area_gdf
    # )

    # and final result list
    results_df = optimization_execution.create_results_df(
        optimization_result_list, model_list[0], forest_area_gdf
    )

    return results_df, optimization_result_list, model_list


results_df, optimization_result_list, model_list = optimization_main_loop(forest_area_3)

optimization_results = [results_df, optimization_result_list, model_list]

/home/carlo/Documents/Thinkpad Backup/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


#### Plotting


In [ ]:
(
    interactive_layout,
    current_cable_roads_table_figure,
    layout_overview_table_figure,
    anchor_table_figure,
    road_anchor_table_figure,
    pareto_frontier,
    move_left_button,
    move_right_button,
    reset_all__CRs_button,
    add_to_layout_button,
    reset_comparison_button,
    view_in_3d_button,
    dropdown_menu,
    layout_comparison_figure,
    layout_3d_scatter_plot,
    explanation_widget,
    explanation_dropdown_menu,
) = interactive_cr_selection(
    forest_area_3, model_list, optimization_result_list, results_df
)


interface = VBox(
    [
        HBox(
            [
                move_left_button,
                move_right_button,
                reset_all__CRs_button,
                add_to_layout_button,
                reset_comparison_button,
                dropdown_menu,
                view_in_3d_button,
            ]
        ),
        HBox(
            [
                interactive_layout,
                layout_comparison_figure,
            ]
        ),
        HBox(
            [
                VBox(
                    [
                        pareto_frontier,
                        explanation_dropdown_menu,
                        explanation_widget,
                    ]
                ),
                VBox(
                    [
                        current_cable_roads_table_figure,
                        layout_overview_table_figure,
                        road_anchor_table_figure,
                    ]
                ),
            ]
        ),
        layout_3d_scatter_plot,
    ]
)

display(interface)

test
